In [ ]:
# 导入必要的库
import numpy as np
import torch
import torchvision.transforms as T
from decord import VideoReader, cpu
from PIL import Image
from torchvision.transforms.functional import InterpolationMode
from modelscope import AutoModel, AutoTokenizer,TextIteratorStreamer
import os
from threading import Thread

# 模型配置
model_path = '/root/autodl-tmp/models/OpenGVLab/InternVL3_5-8B'

# 初始化分词器和模型 - 使用自动多卡分配
print(f"可用GPU数量: {torch.cuda.device_count()}")
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
# 使用 AutoModel 类的 from_pretrained 方法从指定路径加载预训练模型
model = AutoModel.from_pretrained(
    model_path,
    dtype=torch.bfloat16,
    load_in_8bit=False,
    low_cpu_mem_usage=True,
    use_flash_attn=True,
    trust_remote_code=True,
    device_map="auto").eval()
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True, use_fast=False)
# ImageNet 数据集的均值和标准差，用于图像归一化处理。在深度学习中，对图像数据进行归一化可以加速模型收敛，
# 并提升模型的稳定性和泛化能力。这里的均值和标准差是 ImageNet 大规模图像数据集上计算得到的统计值，
# 后续会在图像预处理流程中使用这些值对输入图像进行标准化操作。
# IMAGENET_MEAN 中的三个值分别表示 ImageNet 数据集中图像在 RGB 三个通道上的均值
IMAGENET_MEAN = (0.485, 0.456, 0.406)
# IMAGENET_STD 中的三个值分别表示 ImageNet 数据集中图像在 RGB 三个通道上的标准差
IMAGENET_STD = (0.229, 0.224, 0.225)

def build_transform(input_size):
    """
    构建一个用于对输入图像进行预处理的转换流水线，包含图像格式转换、尺寸调整、转换为张量以及归一化等操作。
    
    参数:
        input_size: 输入图像大小
    
    返回:
        transform: 转换pipeline
    """
    # 从全局变量中获取 ImageNet 数据集的均值和标准差，用于后续图像归一化处理
    MEAN, STD = IMAGENET_MEAN, IMAGENET_STD
    # 使用 torchvision.transforms.Compose 方法构建一个图像转换流水线，按顺序执行一系列图像转换操作
    transform = T.Compose([
        # 使用 Lambda 表达式检查图像模式，如果图像模式不是 RGB，则将其转换为 RGB 模式
        T.Lambda(lambda img: img.convert("RGB") if img.mode != "RGB" else img), 
        # 将图像调整为指定大小 (input_size, input_size)，使用双三次插值方法
        T.Resize((input_size, input_size), interpolation=InterpolationMode.BICUBIC), 
        # 将 PIL 图像或 NumPy 数组转换为 PyTorch 张量
        T.ToTensor(), 
        # 使用 ImageNet 的均值和标准差对图像张量进行归一化处理，加速模型收敛并提升稳定性，每个通道分别减去均值并除以标准差
        # 归一化处理可以将图像的像素值缩放到 [-1, 1] 范围内，这有助于模型训练的稳定性和收敛速度
        T.Normalize(mean=MEAN, std=STD)
    ])
    return transform


def find_closest_aspect_ratio(aspect_ratio, target_ratios, width, height, image_size):
    """
    寻找最接近原始图像宽高比的目标比例
    
    参数:
        aspect_ratio: 原始图像的宽高比
        target_ratios: 目标比例列表
        width: 原始图像宽度
        height: 原始图像高度
        image_size: 目标图像大小
        
    返回:
        best_ratio: 最佳比例
    """
    # 初始化最佳比例差异为正无穷大，用于后续比较时能找到更小的差异值，inf 表示正无穷大
    # 初始化最佳比例为 (1, 1)，作为初始的默认最佳比例
    best_ratio_diff = float("inf")
    best_ratio = (1, 1)
    # 计算原始图像的面积，后续用于在比例差异相同时进行进一步的筛选
    # 原始图像面积 = 宽度 * 高度
    area = width * height

    # 遍历目标比例列表，寻找最接近原始图像宽高比的目标比例
    for ratio in target_ratios:
        # 计算当前目标比例对应的宽高比
        target_aspect_ratio = ratio[0] / ratio[1]
        # 计算原始图像宽高比与当前目标宽高比的差异绝对值
        ratio_diff = abs(aspect_ratio - target_aspect_ratio)

        # 如果当前比例差异小于最佳比例差异，更新最佳比例差异和最佳比例
        if ratio_diff < best_ratio_diff:
            best_ratio_diff = ratio_diff
            best_ratio = ratio
        # 如果当前比例差异等于最佳比例差异，进行进一步筛选
        elif ratio_diff == best_ratio_diff:
            # 如果原始图像面积大于目标图像面积的一半（目标图像面积由目标比例和图像尺寸计算得出），更新最佳比例
            if area > 0.5 * image_size * image_size * ratio[0] * ratio[1]:
                best_ratio = ratio

    # 返回找到的最接近原始图像宽高比的目标比例
    return best_ratio


def dynamic_preprocess(image, min_num=1, max_num=6, image_size=448, use_thumbnail=False):
    """
    动态预处理图像，根据宽高比将图像分割成多个块
    
    参数:
        image: 原始图像
        min_num: 最小块数
        max_num: 最大块数
        image_size: 目标图像大小
        use_thumbnail: 是否使用缩略图
        
    返回:
        processed_images: 处理后的图像列表
    """
    orig_width, orig_height = image.size
    aspect_ratio = orig_width / orig_height

    # 计算现有图像宽高比
    target_ratios = set((i, j) for n in range(min_num, max_num + 1) for i in range(1, n + 1) for j in range(1, n + 1) if i * j <= max_num and i * j >= min_num)
    target_ratios = sorted(target_ratios, key=lambda x: x[0] * x[1])

    # 寻找最接近目标的宽高比
    target_aspect_ratio = find_closest_aspect_ratio(aspect_ratio, target_ratios, orig_width, orig_height, image_size)

    # 计算目标宽度和高度
    target_width = image_size * target_aspect_ratio[0]
    target_height = image_size * target_aspect_ratio[1]
    blocks = target_aspect_ratio[0] * target_aspect_ratio[1]

    # 调整图像大小
    resized_img = image.resize((target_width, target_height))
    processed_images = []
    for i in range(blocks):
        box = ((i % (target_width // image_size)) * image_size, (i // (target_width // image_size)) * image_size, 
               ((i % (target_width // image_size)) + 1) * image_size, ((i // (target_width // image_size)) + 1) * image_size)
        # 分割图像
        split_img = resized_img.crop(box)
        processed_images.append(split_img)
    assert len(processed_images) == blocks
    if use_thumbnail and len(processed_images) != 1:
        thumbnail_img = image.resize((image_size, image_size))
        processed_images.append(thumbnail_img)
    return processed_images


def load_image(image, input_size=448, max_num=6):
    """
    加载并处理图像
    
    参数:
        image: 输入图像
        input_size: 输入大小
        max_num: 最大块数
        
    返回:
        pixel_values: 处理后的图像张量
    """
    transform = build_transform(input_size=input_size)
    images = dynamic_preprocess(image, image_size=input_size, use_thumbnail=True, max_num=max_num)
    pixel_values = [transform(image) for image in images]
    # 使用 torch.stack 方法将多个图像张量合并为一个批次张量。
    # pixel_values 是一个包含多个图像张量的列表，每个张量代表经过预处理后的单张图像。
    # torch.stack 会在新的维度上对这些张量进行拼接，创建一个新的批次张量。
    # 例如，如果列表中有 N 个形状为 (C, H, W) 的图像张量，经过 stack 操作后，
    # 输出张量的形状将变为 (N, C, H, W)，方便后续模型批量处理。
    pixel_values = torch.stack(pixel_values)
    return pixel_values


def get_index(bound, fps, max_frame, first_idx=0, num_segments=32):
    """
    获取视频帧索引
    
    参数:
        bound: 时间边界 [开始时间, 结束时间]
        fps: 视频帧率
        max_frame: 最大帧数
        first_idx: 第一帧索引
        num_segments: 分段数量
        
    返回:
        frame_indices: 帧索引数组
    """
    if bound:
        start, end = bound[0], bound[1]
    else:
        start, end = -100000, 100000
    start_idx = max(first_idx, round(start * fps))
    end_idx = min(round(end * fps), max_frame)
    seg_size = float(end_idx - start_idx) / num_segments
    frame_indices = np.array([int(start_idx + (seg_size / 2) + np.round(seg_size * idx)) for idx in range(num_segments)])
    return frame_indices

def get_num_frames_by_duration(duration):
    """
    根据视频时长计算帧数
    
    参数:
        duration: 视频时长（秒）
        
    返回:
        num_frames: 计算出的帧数
    """
    local_num_frames = 4        
    num_segments = int(duration // local_num_frames)
    if num_segments == 0:
        num_frames = local_num_frames
    else:
        num_frames = local_num_frames * num_segments
    
    num_frames = min(512, num_frames)
    num_frames = max(128, num_frames)

    return num_frames

def load_video(video_path, bound=None, input_size=448, max_num=1, num_segments=32, get_frame_by_duration = False):
    """
    加载并处理视频
    
    参数:
        video_path: 视频路径
        bound: 时间边界
        input_size: 输入大小
        max_num: 最大块数
        num_segments: 分段数量
        get_frame_by_duration: 是否根据时长获取帧数
        
    返回:
        pixel_values: 处理后的视频帧张量
        num_patches_list: 每帧的块数列表
    """
    vr = VideoReader(video_path, ctx=cpu(0), num_threads=10) # 增加线程数加速读取
    max_frame = len(vr) - 1
    fps = float(vr.get_avg_fps())

    pixel_values_list, num_patches_list = [], []
    transform = build_transform(input_size=input_size)
    if get_frame_by_duration:
        duration = max_frame / fps
        num_segments = get_num_frames_by_duration(duration)
    frame_indices = get_index(bound, fps, max_frame, first_idx=0, num_segments=num_segments)
    for frame_index in frame_indices:
        img = Image.fromarray(vr[frame_index].asnumpy()).convert("RGB")
        img = dynamic_preprocess(img, image_size=input_size, use_thumbnail=True, max_num=max_num)
        pixel_values = [transform(tile) for tile in img]
        pixel_values = torch.stack(pixel_values)
        num_patches_list.append(pixel_values.shape[0])
        pixel_values_list.append(pixel_values)
    # 将存储每帧图像张量的列表 pixel_values_list 中的所有张量在第 0 维（批次维度）上进行拼接。
    # 在 load_video 函数中，pixel_values_list 存储了视频中每个采样帧经过预处理和分块后的图像张量，
    # 每个张量的形状可能为 (num_patches, C, H, W)，其中 num_patches 表示该帧图像被分割成的块数，
    # C 是通道数，H 和 W 是图像的高度和宽度。通过 torch.cat 操作，将所有帧的图像块张量拼接成一个大的张量，
    # 最终得到的 pixel_values 张量形状为 (total_patches, C, H, W)，方便后续模型对所有图像块进行批量处理。
    # 处理后的视频帧批量移动到GPU，减少设备间传输次数
    pixel_values = torch.cat(pixel_values_list).to(model.device, non_blocking=True)
    # 处理后的视频帧批量移动到GPU，减少设备间传输次数
    # num_patches_list = torch.tensor(num_patches_list, dtype=torch.long, device=model.device, non_blocking=True)
    return pixel_values, num_patches_list

In [ ]:
import gc
# 评估设置
# max_num_frames = 64
generation_config = dict(
    do_sample=False,  # 启用采样，使用贪婪解码
    temperature=0.0,  # 温度参数，设置为0.0表示禁用采样
    max_new_tokens=1024,  # 最大新生成令牌数
    top_p=0.1,  #  nucleus sampling 概率阈值
    num_beams=1,  # 束搜索束数
)

video_path = "car.mp4"
num_segments=8  # 视频片段数，如果机器显存不够就减小当前值

# torch.no_grad() 来禁用梯度计算
with torch.no_grad():
  # 清理GPU缓存
  # torch.cuda.empty_cache()
  # gc.collect()
  # torch.cuda.reset_peak_memory_stats()

  # 查看当前内存使用情况
  # print(f"当前GPU内存使用: {torch.cuda.memory_allocated()/1024**3:.2f} GiB")
  # print(f"最大GPU内存使用: {torch.cuda.max_memory_allocated()/1024**3:.2f} GiB")
    
  # 启用自动混合精度
  # 加载视频并处理
  pixel_values, num_patches_list = load_video(video_path, num_segments=num_segments, max_num=1, get_frame_by_duration=False)
  pixel_values = pixel_values.to(torch.bfloat16).to(model.device)
  video_prefix = "".join([f"Frame{i+1}: <image>\n" for i in range(len(num_patches_list))])
  
  # 单轮对话：视频详细描述
  question1 = "Describe this video in detail."
  question = video_prefix + question1
  output1, chat_history = model.chat(tokenizer, pixel_values, question, generation_config, num_patches_list=num_patches_list, history=None, return_history=True)
  print(output1)
  
  # 多轮对话：询问视频中的人数
  question2 = "How many people appear in the video?"
  output2, chat_history = model.chat(tokenizer, pixel_values, question2, generation_config, num_patches_list=num_patches_list, history=chat_history, return_history=True)
  
  print(output2)

In [ ]:
with torch.no_grad():
  # 单轮对话：询问车辆损伤部位（中文）
  question1 = "车的哪个部位损伤了？"
  question = video_prefix + question1
  output1, chat_history = model.chat(tokenizer, pixel_values, question, generation_config, num_patches_list=num_patches_list, history=None, return_history=True)
  print(output1)
  
  # 多轮对话：询问车辆碰撞位置（中文）
  question2 = "车撞到哪里了？"
  output2, chat_history = model.chat(tokenizer, pixel_values, question2, generation_config, num_patches_list=num_patches_list, history=chat_history, return_history=True)
  
  print(output2)

In [ ]:
# 采用流传输
# Initialize the streamer
streamer = TextIteratorStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True, timeout=10)
# Define the generation configuration
generation_config = dict(max_new_tokens=1024, do_sample=False, streamer=streamer)
video_path = "car.mp4"
num_segments=8  # 视频片段数，如果机器显存不够就减小当前值
with torch.no_grad():
    # 启用自动混合精度
    # 加载视频并处理
    pixel_values, num_patches_list = load_video(video_path, num_segments=num_segments, max_num=1, get_frame_by_duration=False)
    pixel_values = pixel_values.to(torch.bfloat16).to(model.device)
    video_prefix = "".join([f"Frame{i+1}: <image>\n" for i in range(len(num_patches_list))])
    # 单轮对话：询问车辆损伤部位（中文）
    question1 = "车的哪个部位损伤了？"
    question = video_prefix + question1
    # Start the model chat in a separate thread
    thread = Thread(target=model.chat, kwargs=dict(
        tokenizer=tokenizer, pixel_values=pixel_values, question=question,
        history=None, return_history=False, generation_config=generation_config,
    ))
    thread.start()
    
    # Initialize an empty string to store the generated text
    generated_text = ''
    # Loop through the streamer to get the new text as it is generated
    for new_text in streamer:
        if new_text == model.conv_template.sep:
            break
        generated_text += new_text
        print(new_text, end='', flush=True)  # Print each new chunk of generated text on the same line